<a href="https://colab.research.google.com/github/drew-walkerr/Diss_Detecting_Provider_Bias/blob/main/scare_quote_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score




import torch
torch.cuda.empty_cache()
import seaborn as sns
import transformers
import json
from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader
from transformers import RobertaModel, RobertaTokenizer
import logging
logging.basicConfig(level=logging.ERROR)
from google.colab import output
import seaborn as sn
import pandas as pd
import matplotlib.pyplot as plt

#GPU usage setup
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 55.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 30.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 79.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 30.1 MB/s eta 0:00:00


In [ ]:
# mount drive to access data
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:

%cd /content/drive/MyDrive/Diss_Detecting_Provider_Bias/Aim 1/Scare Quotes/3_Classification

%ls

/content/drive/MyDrive/Diss_Detecting_Provider_Bias/Aim 1/Scare Quotes/3_Classification
annotation_Drew_sample_scare_quotes.csv  predictions.csv
gold_standard_scare_quotes_100.csv       scare_quote_classification.ipynb


In [ ]:
df = pd.read_csv("gold_standard_scare_quotes_100.csv")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99 entries, 0 to 98
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   sentence_id  99 non-null     int64 
 1   sentence     99 non-null     object
 2   quoted_text  99 non-null     object
 3   scare_label  99 non-null     int64 
dtypes: int64(2), object(2)
memory usage: 3.2+ KB


In [ ]:

df['label'] = df['scare_label']
df.drop(['sentence_id','scare_label'], axis=1)
del df['sentence_id'], df['scare_label']
df.head(3)

,sentence,quoted_text,label
0,"Pt stated year as ""[**2159**]"" and when asked ...",[**2159**],0
1,Speech evaluation recommended behavioral modif...,break,0
2,"psychosocial: pt tearful several times, mouthi...",go home,0


In [ ]:
cols = ['sentence', 'quoted_text']
df['text'] = df[cols].apply(lambda row: '</s>'.join(row.values.astype(str)), axis=1)
df.head(3)

,sentence,quoted_text,label,text
0,"Pt stated year as ""[**2159**]"" and when asked ...",[**2159**],0,"Pt stated year as ""[**2159**]"" and when asked ..."
1,Speech evaluation recommended behavioral modif...,break,0,Speech evaluation recommended behavioral modif...
2,"psychosocial: pt tearful several times, mouthi...",go home,0,"psychosocial: pt tearful several times, mouthi..."


In [ ]:
MAX_LEN = 512
TRAIN_BATCH_SIZE = 8
VALID_BATCH_SIZE = 8
EPOCHS = 10
LEARNING_RATE = 5e-05
tokenizer = RobertaTokenizer.from_pretrained('roberta-base', truncation=True, do_lower_case=True)

In [ ]:
class BiasData(Dataset):
    def __init__(self, dataframe, tokenizer, max_len):
        self.tokenizer = tokenizer
        self.data = dataframe
        self.text = dataframe.text
        self.targets = self.data.label
        self.max_len = max_len

    def __len__(self):
        return len(self.text)

    def __getitem__(self, index):
        text = str(self.text[index])
        text = " ".join(text.split())

        inputs = self.tokenizer.encode_plus(
            text,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            pad_to_max_length=True,
            return_token_type_ids=True
        )
        ids = inputs['input_ids']
        mask = inputs['attention_mask']
        token_type_ids = inputs["token_type_ids"]

        #print(self.targets.shape)

        return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long),
            'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long),
            'targets': torch.tensor(self.targets[index], dtype=torch.float)
        }

In [ ]:
newdf = df[['text','label']]
newdf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99 entries, 0 to 98
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    99 non-null     object
 1   label   99 non-null     int64 
dtypes: int64(1), object(1)
memory usage: 1.7+ KB


In [ ]:
train_size = 0.8
train_data=newdf.sample(frac=train_size,random_state=0)
test_data=newdf.drop(train_data.index).reset_index(drop=True)
train_data = train_data.reset_index(drop=True)

print("FULL Dataset: {}".format(newdf.shape))
print("TRAIN Dataset: {}".format(train_data.shape))
print("TEST Dataset: {}".format(test_data.shape))

training_set = BiasData(train_data, tokenizer, MAX_LEN)
testing_set = BiasData(test_data, tokenizer, MAX_LEN)

FULL Dataset: (99, 2)
TRAIN Dataset: (79, 2)
TEST Dataset: (20, 2)


In [ ]:
train_data['label'].unique()

array([1, 0])

In [ ]:
test_data['label'].unique()

array([0, 1])

In [ ]:
train_params = {'batch_size': TRAIN_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

test_params = {'batch_size': VALID_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

training_loader = DataLoader(training_set, **train_params)
testing_loader = DataLoader(testing_set, **test_params)

In [ ]:
class RobertaClass(torch.nn.Module):
    def __init__(self):
        super(RobertaClass, self).__init__()
        self.l1 = RobertaModel.from_pretrained("roberta-base")
        self.pre_classifier = torch.nn.Linear(768, 768)
        self.dropout = torch.nn.Dropout(0.0)
        self.classifier = torch.nn.Linear(768, 2) #output labels = 1 for binary classification

    def forward(self, input_ids, attention_mask, token_type_ids):
        output_1 = self.l1(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
        hidden_state = output_1[0]
        pooler = hidden_state[:, 0]
        pooler = self.pre_classifier(pooler)
        pooler = torch.nn.ReLU()(pooler)
        pooler = self.dropout(pooler)
        output = self.classifier(pooler)
        return output

In [ ]:
model = RobertaClass()
model.to(device)

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


RobertaClass(
  (l1): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((

In [ ]:
# Creating the loss function and optimizer
loss_function = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params =  model.parameters(), lr=LEARNING_RATE)

In [ ]:
def calcuate_accuracy(preds, targets):
    n_correct = (preds==targets).sum().item()
    return n_correct

In [ ]:
def train(epoch):
    tr_loss = 0
    n_correct = 0
    nb_tr_steps = 0
    nb_tr_examples = 0
    model.train()
    for _,data in tqdm(enumerate(training_loader, 0)):
        ids = data['ids'].to(device, dtype = torch.long)
        mask = data['mask'].to(device, dtype = torch.long)
        token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
        targets = data['targets'].to(device, dtype = torch.long)

        outputs = model(ids, mask, token_type_ids)
        loss = loss_function(outputs, targets)
        tr_loss += loss.item()
        big_val, big_idx = torch.max(outputs.data, dim=1)
        n_correct += calcuate_accuracy(big_idx, targets)

        nb_tr_steps += 1
        nb_tr_examples+=targets.size(0)

        if _%5000==0:
            loss_step = tr_loss/nb_tr_steps
            accu_step = (n_correct*100)/nb_tr_examples
            print(f"Training Loss: {loss_step}")
            print(f"Training Accuracy: {accu_step}")

        optimizer.zero_grad()
        loss.backward()
        # # When using GPU
        optimizer.step()

    print(f'The Total Accuracy for Epoch {epoch}: {(n_correct*100)/nb_tr_examples}')
    epoch_loss = tr_loss/nb_tr_steps
    epoch_accu = (n_correct*100)/nb_tr_examples
    print(f"Training Loss Epoch: {epoch_loss}")
    print(f"Training Accuracy Epoch: {epoch_accu}")

    return

In [ ]:
for epoch in range(EPOCHS):
    train(epoch)

0it [00:00, ?it/s]Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2418: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


Training Loss: 0.7423888444900513
Training Accuracy: 12.5


10it [00:09,  1.04it/s]


The Total Accuracy for Epoch 0: 49.36708860759494
Training Loss Epoch: 0.6818287134170532
Training Accuracy Epoch: 49.36708860759494


0it [00:00, ?it/s]

...

The Total Accuracy for Epoch 8: 91.13924050632912
Training Loss Epoch: 0.3502341017127037
Training Accuracy Epoch: 91.13924050632912


0it [00:00, ?it/s]

Training Loss: 0.3153141438961029
Training Accuracy: 87.5


10it [00:07,  1.35it/s]

The Total Accuracy for Epoch 9: 96.20253164556962
Training Loss Epoch: 0.12091465573757887
Training Accuracy Epoch: 96.20253164556962


In [ ]:
def calcuate_accuracy(preds, targets):
    n_correct = (preds==targets).sum().item()
    return n_correct

In [ ]:
def valid(model, testing_loader):
    model.eval()
    n_correct = 0
    tr_loss=0
    nb_tr_steps=0
    nb_tr_examples=0

    all_preds = []  # list to store predictions
    all_targets = []  # list to store original targets
    all_texts = []  # list to store original input texts

    with torch.no_grad():
        for _, data in tqdm(enumerate(testing_loader, 0)):
            ids = data['ids'].to(device, dtype = torch.long)
            mask = data['mask'].to(device, dtype = torch.long)
            token_type_ids = data['token_type_ids'].to(device, dtype=torch.long)
            targets = data['targets'].to(device, dtype = torch.long)

            outputs = model(ids, mask, token_type_ids).squeeze()
            loss = loss_function(outputs, targets)
            tr_loss += loss.item()
            big_val, big_idx = torch.max(outputs.data, dim=1)

            all_preds.extend(big_idx.cpu().numpy())  # store predictions
            all_targets.extend(targets.cpu().numpy())  # store targets

            all_texts.extend(data['ids'])  # store original input texts

            n_correct += calcuate_accuracy(big_idx, targets)

            nb_tr_steps += 1
            nb_tr_examples += targets.size(0)

            if _ % 5000 == 0:
                loss_step = tr_loss/nb_tr_steps
                accu_step = (n_correct*100)/nb_tr_examples
                print(f"Validation Loss: {loss_step}")
                print(f"Validation Accuracy: {accu_step}")

    epoch_loss = tr_loss/nb_tr_steps
    epoch_accu = (n_correct*100)/nb_tr_examples

    print(f"Validation Loss Epoch: {epoch_loss}")
    print(f"Validation Accuracy Epoch: {epoch_accu}")

    # Print classification report
    report = classification_report(all_targets, all_preds)
    print(report)

    # Confusion matrix
    cm = confusion_matrix(all_targets, all_preds)
    print("Confusion Matrix:")
    print(cm)

    # Create a DataFrame and save it
    df_predictions = pd.DataFrame({
        'Text': all_texts,
        'Original': all_targets,
        'Predicted': all_preds
    })
    df_predictions.to_csv('predictions.csv', index=False)
    print(df_predictions.head())

    return epoch_accu


Initial Run -- one interation

In [ ]:
valid(model, testing_loader)

acc = valid(model,testing_loader)
print("test accuracy = %0.2f%%" % acc)


1it [00:00,  4.17it/s]

Validation Loss: 1.694406509399414
Validation Accuracy: 62.5


3it [00:00,  4.83it/s]
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Validation Loss Epoch: 2.409546653429667
Validation Accuracy Epoch: 55.0
              precision    recall  f1-score   support

           0       0.55      1.00      0.71        11
           1       0.00      0.00      0.00         9

    accuracy                           0.55        20
   macro avg       0.28      0.50      0.35        20
weighted avg       0.30      0.55      0.39        20

Confusion Matrix:
[[11  0]
 [ 9  0]]
                                                Text  Original  Predicted
0  [tensor(0), tensor(3320), tensor(740), tensor(...         0          0
1  [tensor(0), tensor(12228), tensor(42781), tens...         0          0
2  [tensor(0), tensor(133), tensor(3186), tensor(...         0          0
3  [tensor(0), tensor(510), tensor(90), tensor(14...         0          0
4  [tensor(0), tensor(14693), tensor(7), tensor(3...         1          0


0it [00:00, ?it/s]/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2418: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
1it [00:00,  4.18it/s]

Validation Loss: 2.3658487796783447
Validation Accuracy: 50.0


3it [00:00,  4.88it/s]
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Validation Loss Epoch: 2.3963660399119058
Validation Accuracy Epoch: 55.0
              precision    recall  f1-score   support

           0       0.55      1.00      0.71        11
           1       0.00      0.00      0.00         9

    accuracy                           0.55        20
   macro avg       0.28      0.50      0.35        20
weighted avg       0.30      0.55      0.39        20

Confusion Matrix:
[[11  0]
 [ 9  0]]
                                                Text  Original  Predicted
0  [tensor(0), tensor(32773), tensor(9), tensor(1...         1          0
1  [tensor(0), tensor(14693), tensor(7), tensor(3...         1          0
2  [tensor(0), tensor(8275), tensor(154), tensor(...         0          0
3  [tensor(0), tensor(46577), tensor(40083), tens...         1          0
4  [tensor(0), tensor(133), tensor(3186), tensor(...         0          0
test accuracy = 55.00%


In order to account for variations in GPU states, we ran each of the models 10 times and provide the mean accuracy


## Predictions

In [ ]:
def predict_on_dataframe(df, model, tokenizer, max_len):
    model.eval()
    predictions = []

    for index, row in df.iterrows():
        text = row['text']
        inputs = tokenizer.encode_plus(
            text,
            None,
            add_special_tokens=True,
            max_length=max_len,
            pad_to_max_length=True,
            return_token_type_ids=True
        )
        ids = torch.tensor([inputs['input_ids']], dtype=torch.long).to(device)
        mask = torch.tensor([inputs['attention_mask']], dtype=torch.long).to(device)
        token_type_ids = torch.tensor([inputs["token_type_ids"]], dtype=torch.long).to(device)

        with torch.no_grad():
            outputs = model(ids, mask, token_type_ids)

        big_val, big_idx = torch.max(outputs.data, dim=1)
        predictions.append(big_idx[0].item())

    df['predictions'] = predictions
    return df


In [ ]:
predicted_classes = predict_on_dataframe(newdf, model, tokenizer, max_len=128)
print(predicted_classes)

annotation_drew_sample = pd.read_csv("annotation_Drew_sample_scare_quotes.csv")
annotation_drew_sample.info()
annotation_drew_sample= annotation_drew_sample.rename(columns={'Sentence':'text'})

predicted_mega = predict_on_dataframe(annotation_drew_sample, model, tokenizer, max_len=128)
predicted_mega
#
filtered_df = predicted_mega[predicted_mega['predictions'] == 1]

filtered_df

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2418: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
<ipython-input-23-bbb1cbe3f7d1>:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['predictions'] = predictions


                                                 text  label  predictions
0   Pt stated year as "[**2159**]" and when asked ...      0            0
1   Speech evaluation recommended behavioral modif...      0            0
2   psychosocial: pt tearful several times, mouthi...      0            0
3   Nursing Progress Note: DNR/DNI, no NRB, no bip...      0            0
4   NEURO: Tylenol only for pain, pt states percoc...      1            1
..                                                ...    ...          ...
94  Chief Complaint: 24 Hour Events: - ICU consent...      1            0
95  In view of patient's history of remote right l...      0            0
96  pt recieved 50ml of antibiotic cipro when pt c...      0            0
97  Nursing Assessment Note 1900-0700 NEURO: Pt al...      0            0
98  Metoprolol was stopped due to side effects (pa...      1            1

[99 rows x 3 columns]
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (tot

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2418: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


,Unnamed: 0,Sentence ID,ROW_ID_x,SUBJECT_ID,HADM_ID,CHARTDATE,CHARTTIME,STORETIME,CATEGORY,DESCRIPTION,...,DOB,DOD,DOD_HOSP,DOD_SSN,EXPIRE_FLAG,text,quoted_text,scare_quote,annotator_comments,predictions
39,10491216,10491216,3241,22754,101703.0,2119-04-19,NaN,NaN,Discharge summary,Report,...,2037-06-17,2119-11-29,2119-11-29,2119-11-29,1,The patient's daughter reports that the patie...,a while,NaN,NaN,1
124,13805414,13805414,1261163,65,143430.0,2132-08-10,2132-08-10 18:50:00,2132-08-10 19:19:00,Nursing/other,Report,...,2106-01-17,NaN,NaN,NaN,0,"Per parents, pt does not drink during the wee...",alot,NaN,NaN,1
261,16047838,16047838,1330015,5183,144799.0,2196-11-28,2196-11-28 16:41:00,2196-11-28 16:45:00,Nursing/other,Report,...,2170-07-26,NaN,NaN,NaN,0,"They report that they enjoyed their ""time off""...",time off,NaN,NaN,1
266,12424723,12424723,1338258,5806,194389.0,2156-01-06,2156-01-06 12:29:00,2156-01-06 12:32:00,Nursing/other,Report,...,2099-08-18,2161-01-30,2161-01-30,2161-01-30,1,"Sister said that now they just need to ""wait ...",wait and see,NaN,NaN,1
326,2967196,2967196,12575,23666,170289.0,2112-12-28,NaN,NaN,Discharge summary,Report,...,1812-12-19,2119-11-03,NaN,2119-11-03,1,"The patient reports that she has ""been around...",been around many sick people.,NaN,NaN,1
349,9691618,9691618,683311,57260,158142.0,2113-07-05,2113-07-05 00:52:00,2113-07-05 11:37:27,Physician,Physician Resident Progress Note,...,2030-01-12,2114-07-28,NaN,2114-07-28,1,"# Anemia - baseline HCT unknown, per family, ...",bleeding from GI system,NaN,NaN,1
353,15785973,15785973,53081,75798,149915.0,2186-04-04,NaN,NaN,Discharge summary,Report,...,2138-12-13,NaN,NaN,NaN,0,The patient used to use cocaine and believes t...,sharing straws.,NaN,NaN,1


In [ ]:

# constant for classes
#classes = ('Neither Source/Location', 'Source, no user', 'User, no source', 'User and source')


#print(test_data["label"])
#predicted_roberta = model.
#predict(test_data["text"])
#print(predicted)
# Build confusion matrix
#cf_matrix = confusion_matrix(test_data["label"], predicted)
#df_cm = pd.DataFrame(cf_matrix / np.sum(cf_matrix, axis=1)[:, None], index = [i for i in classes],
#                     columns = [i for i in classes])
#plt.figure(figsize = (4,4))
#sn.heatmap(df_cm, annot=True)
#plt.savefig('output.png')

Notes for next time-- want to turn this script into a function where I can pass through the hyperparameters.

In [ ]:
# svm baseline

#from sklearn.pipeline import make_pipeline
#from sklearn.preprocessing import StandardScaler
#from sklearn.feature_extraction.text import CountVectorizer
#from sklearn.feature_extraction.text import TfidfTransformer
#from sklearn.svm import SVC

#from sklearn.pipeline import Pipeline
#text_clf = Pipeline([('vect', CountVectorizer()), ('tfidf', TfidfTransformer()),('clf', SVC(gamma='auto'))])
#text_clf.fit(train_data['text'], train_data['label'])

In [ ]:
#docs_test = test_data['text']
#predicted = text_clf.predict(docs_test)
#np.mean(predicted == test_data['label'])

In [ ]:
#test_accs = []
#for i in range(10):
#  model = RobertaClass()
#  model.to(device)
#
#  loss_function = torch.nn.CrossEntropyLoss()
#  optimizer = torch.optim.Adam(params =  model.parameters(), lr=LEARNING_RATE)
#
#  for epoch in range(EPOCHS):
#      train(epoch)
#  acc = valid(model, testing_loader)
#  #print("test accuracy = %0.2f%%" % acc)
#  test_accs.append(acc)

# mean_acc = sum(test_accs) / len(test_accs)
# print("MEAN ACCURACY = %0.2f%%" % mean_acc)
# output.eval_js('new Audio("https://upload.wikimedia.org/wikipedia/commons/d/d9/Wilhelm_Scream.ogg").play()')
